# Model 3 Capstone Project :  RAG (Retrieval-Augmented Generation) WasteSnap 
### **Anggota Tim Machine Learning :**
Arthur Setiawan Waruwu | MC319D5Y2042 | Universitas Sumatera Utara  <br>
Sakifa Indira Putri | MC319D5X2380 | Universitas Sumatera Utara  <br>
Diva Anggreini Harahap | MC319D5X2329 | Universitas Sumatera Utara 

#### **Import Library**

In [9]:
import os
import logging
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [12]:
# setup logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)
logger = logging.getLogger(__name__)

pdf_folder = 'DatasetPDF'
all_documents = []

#### **PDF Loader**

In [3]:
pdf_folder = 'DatasetPDF'
all_documents = []

for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        file_path = os.path.join(pdf_folder, filename)
        try:
            print(f"Memuat: {filename}")
            loader = PyMuPDFLoader(file_path)
            documents = loader.load()
            all_documents.extend(documents)
        except Exception as e:
            print(f"Gagal load {filename}: {e}")


logging.getLogger("pypdf").setLevel(logging.ERROR)
print(f"Total halaman yang berhasil dimuat: {len(all_documents)}")
print(f"Total file PDF yang berhasil diproses: {len(set([doc.metadata['source'] for doc in all_documents]))}")


Memuat: ampas-kopi-waste.pdf
Memuat: Analisis Limbah Sapi yang Berpotensi Mencemari Lingkungan dan Menularkan Penyakit pada Masyarakat.pdf
Memuat: buah-waste.pdf
Memuat: cangkang-telur.pdf
Memuat: Dampak Karet.pdf
Memuat: e-waste-2.pdf
Memuat: e-waste-3.pdf
Memuat: e-waste-4.pdf
Memuat: e-waste-5.pdf
Memuat: e-waste-sampah.pdf
Memuat: e-waste.pdf
Memuat: Edukasi Pemanfaatan Ampas Kopi .pdf
Memuat: Elemen Interior Berbahan Baku Pengolahan Sampah Styrofoam dan Sampah Kulit Jeruk.pdf
Memuat: ewasete.pdf
Memuat: IDENTIFIKASI KARAKTERISTIK SAMPAH ELEKTRONIK.pdf
Memuat: IMPLEMENTASI RECYCLE LIMBAH KAYU HASIL PRODUKSI PLYWOOD DAN WORK WOOD MENJADI BAHAN BAKU PRODUK PARTICLE BOARD DI PT.pdf
Memuat: kaca-waste-2.pdf
Memuat: kaca-waste.pdf
Memuat: kain-3.pdf
Memuat: kain.pdf
Memuat: kalenglogam.pdf
Memuat: KARAKTERISTIK SAMPAH ELEKTRONIK.pdf
Memuat: kardus-2.pdf
Memuat: kardus-waste.pdf
Memuat: kardus-waste77.pdf
Memuat: kardus3.pdf
Memuat: Karduss.pdf
Memuat: karet-2.pdf
Memuat: KARET-WASTE3.pd

#### **Text Chunking**
Chuncking per Paragraf

In [11]:
from langchain.text_splitter import CharacterTextSplitter

paragraph_splitter = CharacterTextSplitter(
    separator="\n\n",         
    chunk_size=1000,          
    chunk_overlap=0,          
    keep_separator=False      
)

split_documents = paragraph_splitter.split_documents(all_documents)
print(f"Total chunks paragraf: {len(split_documents)}")

# Tampilkan 5 chunk pertama untuk verifikasi
for i, doc in enumerate(split_documents[:5]):
    print(f"\n--- Chunk {i+1} ---")
    print(f"Source file: {doc.metadata['source']}")
    print(doc.page_content)


Total chunks paragraf: 421

--- Chunk 1 ---
Source file: DatasetPDF\ampas-kopi-waste.pdf
9 
Pemanfaatan Limbah Ampas Kopi menjadi Pupuk Organik  
  
Abstrak  
  
Coffee shop merupakan salah satu tempat populer yang sering didatangi oleh berbagai 
kalangan baik anak muda, dewasa bahkan yang sudah berumur dan menjadi pilihan yang 
nyaman untuk sekedar hangout, mengerjakan tugas atau bahkan mengadakan pertemuan 
klien. Dengan semakin populernya Coffee shop mereka semakin memperbanyak cabang 
dan memiliki banyak tempat dengan nama yang baru. Akibatnya maka akan semakin 
banyak limbah Coffee seperti limbah plastik dan limbah ampas kopi. Oleh karna itu, perlu 
diadakan pengembangan dan pemanfaatan daur ulang sehingga tidak menyebabkan limbah. 
Berbeda dengan kebanyakan Coffee shop, less waste more Coffee mereka memiliki daya 
tarik dengan tidak menggunkan plastik atau 0% plastic dan mendaur ulang ampas kopi yang 
sudah terpakai menjadi pupuk tanaman. Ini merupakan inovasi yang bagus sehingga

#### **Inisialisasi embedding model**

In [13]:
try:
    logger.info("Inisialisasi model embedding HuggingFace...")
    embedding_model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)
    logger.info("Model embedding berhasil diinisialisasi.")
except Exception as e:
    logger.error(f"Error saat inisialisasi embedding: {e}")
    raise

2025-06-02 14:58:14,928 [INFO] Inisialisasi model embedding HuggingFace...
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_5648\821173970.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)
d:\RAG-Project\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-02 14:58:32,715 [INFO] Use pytorch device_name: cpu
2025-06-02 14:58:32,717 [INFO] Load pretrained SentenceTransformer: sentence-transformers/paraphrase-mult

ReadTimeout: (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 06d51bed-58cd-4a85-a0bf-f9b34ff5979c)')

In [11]:
# Coba embed cuma 2 dokumen pertama dari split_documents
sample_texts = [split_documents[0].page_content, split_documents[1].page_content]

embedded_samples = embedding_model.embed_documents(sample_texts)

print(f"Embedding 1 (panjang vektor): {len(embedded_samples[0])}")
print(f"Contoh nilai embedding: {embedded_samples[0][:5]}")  # 5 nilai awal


ValueError: Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002807CA6B710>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))